# 04 — RAII & Smart Pointers: The End of `malloc` 🗑️

## 1. The Core Philosophy: RAII

**RAII** = Resource Acquisition Is Initialization.

The C++ guarantee:
"When an object goes out of scope (end of function, end of loop, or thrown exception), its Destructor is called."

If we acquire memory in the Constructor and release it in the Destructor, **leaks become impossible**.


In [ ]:
#include <iostream>
#include <memory> // Required for smart pointers
#include <string>

// A noisy class to visualize memory management
class Tracker {
    std::string name;
public:
    Tracker(std::string n) : name(n) {
        std::cout << "[Constructed] " << name << std::endl;
    }
    ~Tracker() {
        std::cout << "[Destructed ] " << name << " (Memory Freed)" << std::endl;
    }
    void say_hello() {
        std::cout << "Hello from " << name << std::endl;
    }
};


## 2. `std::unique_ptr` (The Default)

This is your replacement for `malloc`/`free`. 
* **Overhead:** Zero. (It is just a raw pointer wrapper).
* **Semantics:** Exclusive ownership. Only one pointer owns the object.

### The `make_unique` idiom
Don't use `new`. Use `std::make_unique<T>(args)`. It's safer and cleaner.

In [ ]:
std::cout << "--- Scope Start ---" << std::endl;
{
    // OLD C++ way (Avoid): Tracker* t = new Tracker("Old");
    
    // MODERN way: std::make_unique<Type>(arguments...)
    // This creates the object on the heap and wraps it.
    std::unique_ptr<Tracker> ptr = std::make_unique<Tracker>("SmartPtr");

    // Usage: Use it exactly like a raw pointer (-> and * operators are overloaded)
    ptr->say_hello();
    
    // NO delete needed here. Destructor runs automatically at '}'
}
std::cout << "--- Scope End ---" << std::endl;

### ⚠️ Crucial: You cannot copy a unique_ptr
Since it represents **exclusive** ownership, copying it is forbidden. This prevents the "Double Free" bug common in C.

If you want to transfer ownership, you must `std::move()` it.

In [ ]:
void take_ownership(std::unique_ptr<Tracker> t) {
    std::cout << "I now own the pointer!" << std::endl;
} // 't' dies here, so the Tracker is destroyed HERE.

{
    auto ptr = std::make_unique<Tracker>("Mover");
    
    // take_ownership(ptr); // ERROR: Call to implicitly-deleted copy constructor
    
    // std::move casts 'ptr' to an r-value, allowing ownership transfer.
    // 'ptr' is now nullptr.
    take_ownership(std::move(ptr)); 
    
    if (!ptr) std::cout << "ptr is now empty in local scope" << std::endl;
}

## 3. `std::shared_ptr` (Reference Counting)

Use this ONLY when multiple parts of the code need to keep an object alive, and you don't know who will finish last.

* **Mechanism:** Atomic reference counter.
* **Cost:** Slight overhead for thread-safety.

In [ ]:
std::shared_ptr<Tracker> shared1 = std::make_shared<Tracker>("Shared");

{
    std::shared_ptr<Tracker> shared2 = shared1;
    std::cout << "Use count: " << shared1.use_count() << std::endl; // Should be 2
}
// shared2 dies. Count becomes 1. Object is NOT destroyed yet.

std::cout << "Use count: " << shared1.use_count() << std::endl;